In [9]:
import pandas as pd
import numpy as np
from dis import dis
import math
import geopandas as gpd
import pandas as pd
import numpy
from shapely import wkt
from shapely import wkb
from shapely.geometry import Point

modified = pd.read_csv('../data/modified_data.csv')
#modified['geometry'] = [Point(xy) for xy in zip(modified.lon, modified.lat)]


grunnkrets_stripped = pd.read_csv('../data/grunnkrets_norway_stripped.csv')
busstops = pd.read_csv('../data/busstops_norway.csv')
#bs = busstops

df = modified

def dist(lat, lon, df): 
    lat1 = abs(df['lat'] - lat)
    lon1 = abs(df['lon'] - lon)
    eucdist = np.sqrt(lon1**2 + lat1**2)
    print(type(eucdist))
    return eucdist


In [ ]:

def store_dist_lvl(distance):
    distance = 10
    df['num_stores_within_distance_and_same_lvl1'] = 0
    df['num_stores_within_distance_and_same_lvl2'] = 0
    df['num_stores_within_distance'] = 0
    num = 0 

    for index in range(len(df)):
        lat = df._get_value(index,'lat')
        lon = df._get_value(index,'lon')
        lvl1 = df._get_value(index, 'lv1')
        lvl2 = df._get_value(index, 'lv2')
        dist_to_all = dist(lat, lon, df)

        count_distance_and_lvl1 = 0 
        count_distance_and_lvl2 = 0 
        count_distance = 0 #only for distance
        iter = 0
        
        for number in dist_to_all:
            if number < distance:
                count_distance += 1
                if lvl1 == df._get_value(iter, 'lv1'):
                    count_distance_and_lvl1 += 1
                    if lvl2 == df._get_value(iter, 'lv2'):
                        count_distance_and_lvl2 += 1
            iter += 1

        df._set_value(index, 'num_stores_within_distance_and_same_lvl1', count_distance_and_lvl1)
        df._set_value(index, 'num_stores_within_distance_and_same_lvl2', count_distance_and_lvl2)
        df._set_value(index, 'num_stores_within_distance', count_distance)

        num+=1 #for runtime tracking only
        print(str(num) + "/12860")


In [ ]:
store_dist_lvl(0.1)

In [4]:
df.head()

,Unnamed: 0,store_id,grunnkrets_id,lat,lon,mall_dummy,lv1,lv2,total_nbr_people,nbr_people_per_store_in_grunnkrets,counts_gr_lv2,counts_municipality_lv2,mean_revenue_for_municipality_and_level1,mean_revenue_for_municipality_and_level2,mean_revenue_for_municipality_and_level3,revenue
0,0,983540538-974187930-44774,6020303.0,59.743104,10.204928,1.0,1,1.1,157.0,3.140000,28.0,92.0,1.685027,1.657954,2.944334,2.944334
1,1,987074191-973117734-44755,3010306.0,59.913759,10.734031,0.0,1,1.1,NaN,NaN,56.0,924.0,1.684780,1.693765,2.083126,3.211972
2,2,984890265-981157303-64491,6050102.0,60.164751,10.254656,1.0,1,1.1,372.0,26.571429,8.0,34.0,1.600517,1.640187,2.839020,2.839020
3,3,914057442-992924179-126912,18040102.0,67.283669,14.379796,1.0,1,1.1,514.0,10.936170,16.0,62.0,1.738266,1.703086,2.468111,2.331755
4,4,913018583-913063538-668469,16017414.0,63.358068,10.374832,1.0,1,1.1,NaN,NaN,6.0,196.0,1.805180,1.792743,2.427614,1.709826


In [10]:

df['num_busstops_within_distance'] = 0
busstops['lon'] = busstops.geometry.apply(lambda x: x.x)
busstops['lat'] = busstops.geometry.apply(lambda x: x.y)
for index in range(len(df)):
    print(df._get_value(index, 'lat'))
    one_to_all = dist(df._get_value(index, 'lat'), df._get_value(index, 'lon'), busstops)
    one_to_all = one_to_all.to_frame()
    one_to_all.rename( columns={0 :'a'}, inplace=True)
    count = (one_to_all['a'] > 0.1).sum()
    df._set_value(index, 'num_busstops_within_distance', count)



AttributeError: 'str' object has no attribute 'x'

In [ ]:
df.to_csv('test.csv')
df.dtypes